#**Titanic - Machine Learning from Disaster**

*   Utilizaremos os datasets disponíveis no [Kaggle Competitions](https://www.kaggle.com/competitions/titanic/overview);
*   A pontuação é a porcentagem de sobreviventes previstos corretamente (Acurácia);

*   **Parte 1** disponível [aqui](https://colab.research.google.com/drive/1MF4oDo24VNhvFihfC0AXUbMUvDvsXCCr#scrollTo=Hn82KfnIwYw6)
    * Resultado da submissão: **0.66746**


# Parte 1

Repetiremos os iniciais passos realizados na **Parte 1**:

In [ ]:
#Importando pandas
import pandas as pd

In [ ]:
#Visualizando base de treino
treino = pd.read_csv("train.csv")
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
#Visualizando base de teste
teste = pd.read_csv("test.csv")
teste.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#Eliminando colunas com alta cardinalidade
treino = treino.drop(['Name','Ticket','Cabin'], axis = 1)
teste = teste.drop(['Name','Ticket','Cabin'], axis = 1)

In [ ]:
#Substituindo idades vazias pela média das idades
treino.loc[treino.Age.isnull(), 'Age'] = treino.Age.mean()
teste.loc[teste.Age.isnull(), 'Age'] = teste.Age.mean()

In [ ]:
#Substituindo vazios pela moda
treino.loc[treino.Embarked.isnull(), 'Embarked'] = treino.Embarked.mode()[0]

In [ ]:
#Substituindo vazios pela média da Fare
teste.loc[teste.Fare.isnull(), 'Fare'] = teste.Fare.mean()

# Parte 2
# **Entendendo as colunas de texto**

In [ ]:
#Verificando colunas de texto
treino.dtypes == 'object'

PassengerId    False
Survived       False
Pclass         False
Sex             True
Age            False
SibSp          False
Parch          False
Fare           False
Embarked        True
dtype: bool

In [ ]:
#Filtrando colunas de texto
treino.columns[treino.dtypes == 'object']

Index(['Sex', 'Embarked'], dtype='object')

In [ ]:
#Verificando valores na coluna sex
treino.Sex.value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

*   Temos apenas 2 valores possíveis, podemos transformar, em uma nova coluna, em dados numéricos/binários

In [ ]:
#Usando lambda para criação da nova coluna com valores que diferenciam o gênero
treino['Genero'] = treino.Sex.apply(lambda x:1 if x == 'male' else 0)

In [ ]:
#Verificando alteração realizada
treino[['Sex', 'Genero']].head()

,Sex,Genero
0,male,1
1,female,0
2,female,0
3,female,0
4,male,1


In [ ]:
treino[['Sex', 'Genero']].value_counts()

Sex     Genero
male    1         577
female  0         314
Name: count, dtype: int64

In [ ]:
#Realizando o mesmo tratamento na base de teste
teste['Genero'] = teste.Sex.apply(lambda x:1 if x == 'male' else 0)
teste[['Sex', 'Genero']].value_counts()

Sex     Genero
male    1         266
female  0         152
Name: count, dtype: int64

In [ ]:
#Verificando valores na coluna Embarked
treino.Embarked.value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

*   Temos 3 valores distintos, se a transforção for para numérico, existe a possibilidade de o modelo reconhecer os dados como ordenação (1,2,3...).
*   Faremos o tratamento dos dados separando cada um em sua coluna própria com o [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), que possibilita a criação de novas colunas com preenchimento binário.

In [ ]:
#Importação
from sklearn.preprocessing import OneHotEncoder

#Criando encoder
encoder = OneHotEncoder(handle_unknown='ignore',dtype='int32')

#Fit com os dados
encoder.fit(treino[['Embarked']])

#Transformando dados
encoder.transform(treino[['Embarked']]).toarray()

#Transformando resultado em DF
encoder_df = pd.DataFrame(encoder.transform(treino[['Embarked']]).toarray(),columns = encoder.get_feature_names_out())
encoder_df.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [ ]:
#Concatenando na base treino as colunas criadas
treino = pd.concat([treino, encoder_df],axis=1)

In [ ]:
#Verificando alteração realizada
treino[['Embarked', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             646
C         1           0           0             168
Q         0           1           0              77
Name: count, dtype: int64

In [ ]:
#Realizando o mesmo tratamento na base de teste
#   - Posso usar o mesmo fit do treino, não é necessário um novo para a base teste
encoder_df_teste = pd.DataFrame(encoder.transform(teste[['Embarked']]).toarray(),columns = encoder.get_feature_names_out())
teste = pd.concat([teste, encoder_df_teste],axis=1)
teste[['Embarked', 'Embarked_C', 'Embarked_Q', 'Embarked_S']].value_counts()

Embarked  Embarked_C  Embarked_Q  Embarked_S
S         0           0           1             270
C         1           0           0             102
Q         0           1           0              46
Name: count, dtype: int64

In [ ]:
#Eliminando colunas tratadas
treino = treino.drop(['Sex','Embarked'],axis=1)
teste = teste.drop(['Sex','Embarked'],axis=1)

**Criação de modelos para classificação:**
*   1 - Precisamos dividir a base de treino em treino e validação com o [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);
*   2 - Vamos usar os seguintes classificadores:
    * [Árvore de Classificação](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html);
    * [K Vizinhos mais Próximos](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html);
    * [Regressão Logística](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html);

In [ ]:
#Importar train_test_split
from sklearn.model_selection import train_test_split

#Separando base de treino em X e Y
X = treino.drop(['PassengerId','Survived'],axis=1)
y = treino.Survived

#Separando em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.33,random_state=42)

In [ ]:
#Árvore de classificação:

#Importação
from sklearn import tree

#Criando cassificador
clf_ac = tree.DecisionTreeClassifier(random_state=42)

#Fit com os dados
clf_ac = clf_ac.fit(X_train, y_train)

#Realizando previsão
y_pred_ac = clf_ac.predict(X_val)

In [ ]:
#K Vizinhos mais próximos:

#Importação
from sklearn.neighbors import KNeighborsClassifier

#Criando cassificador
clf_knn = KNeighborsClassifier(n_neighbors=3)

#Fit com os dados
clf_knn = clf_knn.fit(X_train, y_train)

#Realizando previsão

y_pred_knn = clf_knn.predict(X_val)

In [ ]:
#Regressão logística

#Importação
from sklearn.linear_model import LogisticRegression

#Criando cassificador
clf_rl = LogisticRegression(random_state=42,max_iter=1000)

#Fit com os dados
clf_rl = clf_rl.fit(X_train, y_train)

#Realizando previsão
y_pred_rl = clf_rl.predict(X_val)

**Avaliando os modelos por:**
*   1 - [Acurácia](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)
*   2 - [Matriz de confusão](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
#Importação Acurácia:
from sklearn.metrics import accuracy_score

In [ ]:
#Árvore de classificação
accuracy_score(y_val, y_pred_ac)

0.7491525423728813

In [ ]:
#K Vizinhos mais próximos
accuracy_score(y_val, y_pred_knn)

0.7152542372881356

In [ ]:
#Regressão Logística
accuracy_score(y_val, y_pred_rl)

0.8169491525423729

In [ ]:
#Importação Matriz de Confusão
from sklearn.metrics import confusion_matrix

In [ ]:
#Árvore de classificação
confusion_matrix(y_val, y_pred_ac)

array([[138,  37],
       [ 37,  83]])

In [ ]:
#K Vizinhos mais próximos
confusion_matrix(y_val, y_pred_knn)

array([[147,  28],
       [ 56,  64]])

In [ ]:
#Regressão Logística
confusion_matrix(y_val, y_pred_rl)

array([[153,  22],
       [ 32,  88]])

**Fazendo previsão na base teste:**
*   Usaremos o modelo com melhor precisão para fazer a previsão (predict)

In [ ]:
#Visualizando o X_train
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,Genero,Embarked_C,Embarked_Q,Embarked_S
6,1,54.000000,0,0,51.8625,1,0,0,1
718,3,29.699118,0,0,15.5000,1,0,1,0
685,2,25.000000,1,2,41.5792,1,1,0,0
73,3,26.000000,1,0,14.4542,1,1,0,0
882,3,22.000000,0,0,10.5167,0,0,0,1


In [ ]:
##Visualizando somente as colunas com valores numéricos da base teste
teste.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Genero,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,1,0,1,0
1,893,3,47.0,1,0,7.0000,0,0,0,1
2,894,2,62.0,0,0,9.6875,1,0,1,0
3,895,3,27.0,0,0,8.6625,1,0,0,1
4,896,3,22.0,1,1,12.2875,0,0,0,1


In [ ]:
#Eliminar coluna ID para que a base teste seja igual a base treino
X_teste = teste.drop(['PassengerId'], axis=1)

In [ ]:
#Aplicando regrassão logística
y_pred = clf_rl.predict(X_teste)

In [ ]:
#Criando coluna com a previsão na base teste
teste['Survived'] = y_pred

In [ ]:
#Selecionando colunas de interesse para fazer envio ao Kaggle
envio_kaggle = teste[['PassengerId','Survived']]

In [ ]:
#Exportando .csv
envio_kaggle.to_csv('resultados_parte02.csv',index=False)

Resultado da submissão: **0.76555**